# Importing libraries

In [1]:
import os, glob, json, csv, subprocess, sys, re
from urllib.request import urlopen
from itertools import chain
import urllib.request
from pprint import pprint
from bs4 import BeautifulSoup
from git import *
from subprocess import Popen, PIPE
from os import path
from collections import Counter
import pandas as pd
import plotly.graph_objs as go
import plotly
from plotly.offline import plot, init_notebook_mode, iplot
import itertools

%matplotlib inline

# Defining directories

In [2]:
userhome = os.path.expanduser('~')
repository = userhome + r'/different-diff/dataset/hbase/'
analyze_dir = userhome + r'/different-diff/analyze/analyze_hbase/'

# Defining git command function

In [3]:
def execute_command(cmd, work_dir):
    #Executes a shell command in a subprocess, waiting until it has completed.
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

# Loading candidate bug-fix commit id

In [43]:
col = ['bug_id','candidate_bugfix_commitID']
bf_cid = pd.read_csv(analyze_dir + '02_extracting_commitid/commitid_of_candidatebugfix.csv')
bf_cid = bf_cid[col]
bf_cid

,bug_id,candidate_bugfix_commitID
0,HBASE-9961,1e7fdb06b2
1,HBASE-9961,560f000424
2,HBASE-9961,e5946ba799
3,HBASE-9958,8a6291a896
4,HBASE-9958,4b84160b37
5,HBASE-9908,f224e4721f
6,HBASE-9908,16b799dae5
7,HBASE-9908,61120fb62c
8,HBASE-9893,c5fc5b993e
9,HBASE-985,e3659aa246


# Extracting files in each candidate bugfix_commitID

In [44]:
len(bf_cid)

1909

Extracting the filename in each candidate bug-fix commit id

In [45]:
file_list = []
for el in range(0,len(bf_cid)):
    sys.stdout.write("\rExtracting bugfix_commitID: {} / {}".format((el+1),len(bf_cid)))
    sys.stdout.flush()
    cmd = "git show --pretty='' --name-only " + bf_cid.iloc[el][1]
    temp = str(execute_command(cmd, repository)).replace("(b'","").split("\\n")[:-1]
    for l, li in enumerate(temp):
        tmp = [bf_cid.iloc[el][0], bf_cid.iloc[el][1], li]
        file_list.append(tmp)
file_list

Extracting bugfix_commitID: 1909 / 1909

[['HBASE-9961',
  '1e7fdb06b2',
  'hbase-common/src/main/java/org/apache/hadoop/hbase/HConstants.java'],
 ['HBASE-9961',
  '560f000424',
  'hbase-common/src/main/java/org/apache/hadoop/hbase/HConstants.java'],
 ['HBASE-9961',
  'e5946ba799',
  'hbase-client/src/main/java/org/apache/hadoop/hbase/client/ClusterStatusListener.java'],
 ['HBASE-9961',
  'e5946ba799',
  'hbase-common/src/main/java/org/apache/hadoop/hbase/HConstants.java'],
 ['HBASE-9958',
  '8a6291a896',
  'src/main/java/org/apache/hadoop/hbase/client/HConnectionManager.java'],
 ['HBASE-9958',
  '4b84160b37',
  'hbase-client/src/main/java/org/apache/hadoop/hbase/HRegionInfo.java'],
 ['HBASE-9958',
  '4b84160b37',
  'hbase-client/src/main/java/org/apache/hadoop/hbase/client/HConnectionManager.java'],
 ['HBASE-9958',
  '4b84160b37',
  'hbase-client/src/main/java/org/apache/hadoop/hbase/client/MetaScanner.java'],
 ['HBASE-9958',
  '4b84160b37',
  'hbase-client/src/main/java/org/apache/hadoop/hbase/protobuf/ProtobufUtil.java'],


Saving into CSV file

In [46]:
with open(analyze_dir + "02_extracting_commitid/file_in_candidatebugfix_commitid.csv", "w") as csvfile:
    fields = ['bug_id', 'candidate_commitID', 'filename']
    writers = csv.writer(csvfile, delimiter = ",")
    writers.writerow(fields)
    for item in file_list:
        writers.writerow(item)
print("File file_in_candidatebugfix_commitid.csv has been created")

File file_in_candidatebugfix_commitid.csv has been created


Extracting the number of files in each candidate bug-fix commit id

In [47]:
file_num = []
for el in range(0,len(bf_cid)):
    sys.stdout.write("\rExtracting bugfix_commitID: {} / {}".format((el+1),len(bf_cid)))
    sys.stdout.flush()
    cmd = "git show --pretty='' --name-only " + bf_cid.iloc[el][1] + " | wc -l"
    temp = str(execute_command(cmd, repository)).replace("(b'","").replace(" ","").split("\\n")[:-1]
    for l, li in enumerate(temp):
        num = [bf_cid.iloc[el][0], bf_cid.iloc[el][1], li]
        file_num.append(num)
file_num

Extracting bugfix_commitID: 1909 / 1909

[['HBASE-9961', '1e7fdb06b2', '1'],
 ['HBASE-9961', '560f000424', '1'],
 ['HBASE-9961', 'e5946ba799', '2'],
 ['HBASE-9958', '8a6291a896', '1'],
 ['HBASE-9958', '4b84160b37', '5'],
 ['HBASE-9908', 'f224e4721f', '1'],
 ['HBASE-9908', '16b799dae5', '1'],
 ['HBASE-9908', '61120fb62c', '11'],
 ['HBASE-9893', 'c5fc5b993e', '2'],
 ['HBASE-985', 'e3659aa246', '2'],
 ['HBASE-9843', '6dd52d7ed3', '7'],
 ['HBASE-984', 'f78f944026', '3'],
 ['HBASE-9786', '230cb08611', '1'],
 ['HBASE-977', '5418547132', '4'],
 ['HBASE-9747', '201c67a7d8', '1'],
 ['HBASE-9747', '3f496372bf', '1'],
 ['HBASE-9747', 'b4df85d6b0', '1'],
 ['HBASE-9747', 'fc15e1ae39', '1'],
 ['HBASE-9747', 'a2a3071256', '1'],
 ['HBASE-9745', '3d37252a32', '1'],
 ['HBASE-9745', '24f150ebbc', '1'],
 ['HBASE-9745', '0bd53be747', '1'],
 ['HBASE-9745', '86033abef2', '1'],
 ['HBASE-9745', 'd289eecd2f', '1'],
 ['HBASE-9745', '8aec79e00c', '1'],
 ['HBASE-9745', 'bcdd3eba60', '1'],
 ['HBASE-9704', '5ff5311da2', '1'],
 ['HBASE-9704', 'abbf77d9cb', 

In [48]:
with open(analyze_dir + "02_extracting_commitid/numberof_file_in_candidatebugfix_commitid.csv", "w") as csvfile:
    fields = ['bug_id', 'candidate_bugfix_commitID', 'number_of_files']
    writers = csv.writer(csvfile, delimiter = ",")
    writers.writerow(fields)
    for n in file_num:
        writers.writerow(n)
print("File numberof_file_in_candidatebugfix_commitid.csv has been created")

File numberof_file_in_candidatebugfix_commitid.csv has been created


In [49]:
df_filenum = pd.read_csv(analyze_dir + "02_extracting_commitid/numberof_file_in_candidatebugfix_commitid.csv")
df_filenum = df_filenum[fields]
df_filenum

,bug_id,candidate_bugfix_commitID,number_of_files
0,HBASE-9961,1e7fdb06b2,1
1,HBASE-9961,560f000424,1
2,HBASE-9961,e5946ba799,2
3,HBASE-9958,8a6291a896,1
4,HBASE-9958,4b84160b37,5
5,HBASE-9908,f224e4721f,1
6,HBASE-9908,16b799dae5,1
7,HBASE-9908,61120fb62c,11
8,HBASE-9893,c5fc5b993e,2
9,HBASE-985,e3659aa246,2


In [50]:
summ = 0
for z in range(0, len(df_filenum)):
    summ = summ + df_filenum.iloc[z][2]
print ("Total number of files for all candidate bug-fix commit id: %i" %summ)

Total number of files for all candidate bug-fix commit id: 9566
